In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats

import matplotlib as mat
import matplotlib.pyplot as plt
mat.style.use('ggplot')

from src.pricer.black_scholes_pricer import*
%load_ext autoreload
%autoreload 2
%matplotlib inline

Is a ATM straddle really no zero delta structure? What needs to be done to obtain a zero delta straddle?

In [2]:
rf = 0.01
T = 365
S = np.array([100])
X = np.array([100])
v = 0.20
d = 0.0
t = 100
op_type = "call"
cpricer = BSPricer(rf,T)
c = cpricer.price(S, X, v, d, t, op_type, True)
c

array([ 4.30692698])

In [3]:
cpricer.print_greeks()

1st order greeks.
Delta:  [ 0.53129024]
Gamma:  [ 0.03799163]
Theta:  [-0.00843943]
Vega:  [ 0.20817331]
Rho:  [ 0.13375917]
2nd order greeks.
Vanna:  [ 0.05204333]
Volga:  [-0.21387669]


In [4]:
op_type = "put"
ppricer = BSPricer(rf,T)
p = ppricer.price(S, X, v, d, t, op_type, True)
p

array([ 4.03332934])

In [5]:
ppricer.print_greeks()

1st order greeks.
Delta:  [-0.46870976]
Gamma:  [ 0.03799163]
Theta:  [-0.00296748]
Vega:  [ 0.20817331]
Rho:  [-0.13946385]
2nd order greeks.
Vanna:  [ 0.05204333]
Volga:  [-0.21387669]


Note that the ATM straddle delta is not equal to zero, which would imply that the delta of the call and put are equal.

In [6]:
atm_straddle_delta = cpricer.delta+ppricer.delta
print("ATM straddle delta: {}".format(atm_straddle_delta[0]))


ATM straddle delta: 0.06258047838141656


This is somewhat obvious as the call option is priced off the foward, where $r$ and $d$ are inputs. In this case, the continuous dividend rate, $d$, is equal to zero, while the risk free rate, $r$, is equal to $1%$ and as a result the forward rate is higher than spot. Thus in order to create a zero delta straddle we would need to adjust the strike higher. Strike (%) of zero delta straddle: $$e^{(r+0.5\sigma^2)T}$$

In [7]:
zero_delta_strike_percentage = np.exp(rf+0.5*(v**2)*(float(t/T)))
zero_delta_strike_percentage

1.0155998793532353

Thus we need to increase the strike by 1.0155998793532353%

In [8]:
rf = 0.01
T = 365
S = np.array([100])
X = np.array([101.01559987935325])
v = 0.20
d = 0.0
t = 100
op_type = "call"
cpricer_zd = BSPricer(rf,T)
c = cpricer_zd.price(S, X, v, d, t, op_type, True)
print(cpricer_zd.delta)
op_type = "put"
ppricer_zd = BSPricer(rf,T)
p = ppricer_zd.price(S, X, v, d, t, op_type, True)
print(ppricer_zd.delta)

[ 0.49281459]
[-0.50718541]


In [9]:
atm_straddle_maybe_zero_delta = cpricer_zd.delta+ppricer_zd.delta
print("ATM straddle delta: {}".format(atm_straddle_maybe_zero_delta[0]))


ATM straddle delta: -0.014370816862032731


Not exactly zero, but the difference can be attributed to model specification and rounding. The general concept holds that an ATM straddle is not zero delta, and a strike shift higher is required, assuming that $rate$ > $dividend$.